In [1]:
import json
import numpy as np
from gurobipy import Model, GRB

In [2]:
with open('instances/toy_instance.json') as f:
    instance = json.load(f)

In [3]:
qualifications_set = set()
for employee in instance['staff']:
    qualifications_set |= set(employee['qualifications'])
qualifications = sorted(qualifications_set)

Nm = len(instance['staff'])
Nc = len(qualifications)
Np = len(instance['jobs'])
Nj = instance['horizon']

In [4]:
HasComp = np.zeros((Nm, Nc), dtype=np.int32)
Conge = np.zeros((Nm, Nj), dtype=np.int32)
NeedComp = np.zeros((Np, Nc), dtype=np.int32)
Rev = np.zeros(Np, dtype=np.int32)
Penalite = np.zeros(Np, dtype=np.int32)
Deadline = np.zeros(Np, dtype=np.int32)

for i, person in enumerate(instance['staff']):
    for qualif in person['qualifications']:
        k = qualifications.index(qualif)
        HasComp[i, k] = 1
    for vacation in person['vacations']:
        Conge[i, vacation-1] = 1
for j, job in enumerate(instance['jobs']):
    for qualif, days in job['working_days_per_qualification'].items():
        k = qualifications.index(qualif)
        NeedComp[j, k] = days
    Rev[j] = job['gain']
    Penalite[j] = job['daily_penalty']
    Deadline[j] = job['due_date']

In [5]:
m = Model("CompuOpti")

# Variables de decision
Travail = m.addMVar((Nm, Np, Nc, Nj), vtype=GRB.BINARY, name="Travail")
ATravail = m.addMVar((Np, Nj), vtype=GRB.BINARY, name="ATravail")
Realise = m.addMVar(Np, vtype=GRB.BINARY, name='Realise')
Debut = m.addMVar(Np, lb=0, ub=Nj-1, vtype=GRB.INTEGER, name='Debut')
Fin = m.addMVar(Np, lb=0, ub=Nj-1, vtype=GRB.INTEGER, name='Fin')
DureeMax = m.addVar(lb=0, ub=Nj,vtype=GRB.INTEGER, name='DureeMax')
AffecteProj = m.addMVar((Nm, Np), vtype=GRB.BINARY, name='AffecteProj')

Retard = Fin - Deadline


# Contrainte de qualification
for j in range(Np):
    for l in range(Nj):
        m.addConstr(Travail[:, j, :, l] <= HasComp)

# Contrainte d’unicité de l’affectation
for i in range(Nm):
    for l in range(Nj):
        m.addConstr(Travail[i, :, :, l].sum() <= 1)

# Contrainte de congé
for j in range(Np):
    for k in range(Nc):
        m.addConstr(Travail[:, j, k, :] <= 1 - Conge)

# Contraintes d’unicité de la réalisation d’un projet et de couverture des qualifications
for j in range(Np):
    for k in range(Nc):
        m.addConstr(Travail[:, j, k, :].sum() == Realise[j] * NeedComp[j, k])

# Contraintes sur la variable ATravail
for i in range(Nm):
    for k in range(Nc):
        m.addConstr(ATravail >= Travail[i, :, k, :])

# Contraintes sur la variable AffectéPro
for k in range(Nc):
    for l in range(Nj):
        m.addConstr(AffecteProj >= Travail[:, :, k, l])

# Contraintes sur la durée d’un projet
for l in range(Nj):
    m.addConstr(Debut <= l * ATravail[:, l] + Nj * (1 - ATravail[:, l]))
    m.addConstr(Fin >= l * ATravail[:, l])
m.addConstr(DureeMax >= Fin - Debut + 1)

# Fonctions objectifs
f1 = -(Rev.T @ Realise - Penalite.T @ Retard)
f2 = AffecteProj.sum()
f3 = DureeMax * 1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-12


In [6]:
m.params.outputflag = 1
m.setObjective(f1, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 985 rows, 281 columns and 1920 nonzeros
Model fingerprint: 0x30515724
Variable types: 0 continuous, 281 integer (270 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -54.0000000
Presolve removed 895 rows and 162 columns
Presolve time: 0.00s
Presolved: 90 rows, 119 columns, 362 nonzeros
Variable types: 0 continuous, 119 integer (114 binary)

Root relaxation: objective -1.164340e+02, 165 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -116

In [11]:
AffecteProj.sum().getValue()

array(15.)

In [12]:
f2_min = 0
f2_max = 15

In [19]:
def find_pareto_fixing_f2(f2_min, f2_max):
    n = 4
    step = int((f2_max - f2_min) / n)
    steps = list(range(int(f2_min),int(f2_max),step)) + [f2_max]

    x1 = []
    for i in steps:
        epsilon = i
        m.params.outputflag = 0
        m.setObjective(f1, GRB.MINIMIZE)
        eps_constr = m.addConstr(f2 == epsilon)
        m.optimize()
        x1.append(None if not hasattr(m, 'ObjVal') else m.ObjVal)
        m.remove(eps_constr)
    return x1

In [20]:
print(find_pareto_fixing_f2(f2_min, f2_max))

[-54.0, -82.0, -91.0, -94.0, -94.0, -94.0]
